<a href="https://colab.research.google.com/github/Nur9702/DeepLearn/blob/main/Qarz_qaytarish_bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

binaryclassificationwithabankchurndataset_path = kagglehub.competition_download('binaryclassificationwithabankchurndataset')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/binaryclassificationwithabankchurndataset/sample_submission.csv
/kaggle/input/binaryclassificationwithabankchurndataset/train.csv
/kaggle/input/binaryclassificationwithabankchurndataset/test.csv


In [ ]:
!/kaggle/input/binaryclassificationwithabankchurndataset

/bin/bash: line 1: /kaggle/input/binaryclassificationwithabankchurndataset: Is a directory


In [ ]:
!/kaggle/input/binaryclassificationwithabankchurndataset/train.csv

/bin/bash: line 1: /kaggle/input/binaryclassificationwithabankchurndataset/train.csv: Permission denied


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [ ]:
train = pd.read_csv('/kaggle/input/binaryclassificationwithabankchurndataset/train.csv', index_col='id')

test = pd.read_csv('/kaggle/input/binaryclassificationwithabankchurndataset/test.csv', index_col='id')

submission = pd.read_csv('/kaggle/input/binaryclassificationwithabankchurndataset/sample_submission.csv')

train.head()

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
id,,,,,,,,,,,,,
0,15702656.0,Nwora,567.0,France,Male,33.0,9.0,0.0,2.0,1.0,0.0,156792.89,0.0
1,15647965.0,Yevdokimova,628.0,France,Female,38.0,3.0,0.0,1.0,1.0,1.0,51987.99,1.0
2,15798834.0,Ch'iu,635.0,France,Female,29.0,3.0,0.0,2.0,1.0,1.0,113079.19,0.0
3,15672056.0,Hsia,681.0,France,Male,28.0,6.0,0.0,2.0,1.0,0.0,14081.64,0.0
4,15759537.0,Okwudilichukwu,587.0,France,Female,27.0,5.0,0.0,2.0,1.0,0.0,158958.90,0.0


In [ ]:
train.isnull().sum()

CustomerId         0
Surname            0
CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

In [ ]:
train.duplicated().sum() #duplikatlarni tekshiriladi

0

In [ ]:
train.Geography.value_counts(), #ustunlar sonini va kachestvasini tekshirish
#train.Gender.value_counts(),
#train.Exited.value_counts()

(Geography
 France     9040
 Spain      3280
 Germany    2680
 Name: count, dtype: int64,)

In [ ]:
train.Gender.value_counts(),

(Gender
 Male      8478
 Female    6522
 Name: count, dtype: int64,)

In [ ]:
test.drop(['CustomerId', 'Surname'], axis=1, inplace=True)
train.drop(['CustomerId', 'Surname'], axis=1, inplace=True) #keraksiz ustunlar o'chirildi

KeyError: "['CustomerId', 'Surname'] not found in axis"

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15000 entries, 0 to 14999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      15000 non-null  float64
 1   Geography        15000 non-null  object 
 2   Gender           15000 non-null  object 
 3   Age              15000 non-null  float64
 4   Tenure           15000 non-null  float64
 5   Balance          15000 non-null  float64
 6   NumOfProducts    15000 non-null  float64
 7   HasCrCard        15000 non-null  float64
 8   IsActiveMember   15000 non-null  float64
 9   EstimatedSalary  15000 non-null  float64
 10  Exited           15000 non-null  float64
dtypes: float64(9), object(2)
memory usage: 1.4+ MB


In [ ]:
train.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,657.856800,37.710133,5.018667,42338.107539,1.590533,0.779133,0.496000,116944.059867,0.198467
std,72.678739,8.144880,2.787407,59703.047751,0.525822,0.414845,0.500001,46047.485455,0.398859
min,431.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,602.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,82644.332500,0.000000
50%,661.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,122449.420000,0.000000
75%,707.000000,42.000000,7.000000,109636.342500,2.000000,1.000000,1.000000,155703.022500,0.000000
max,850.000000,72.000000,10.000000,187911.550000,5.000000,1.000000,1.000000,885120.790000,1.000000


In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [ ]:
train


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
id,,,,,,,,,,,
0,567.0,France,Male,33.0,9.0,0.00,2.0,1.0,0.0,156792.89,0.0
1,628.0,France,Female,38.0,3.0,0.00,1.0,1.0,1.0,51987.99,1.0
2,635.0,France,Female,29.0,3.0,0.00,2.0,1.0,1.0,113079.19,0.0
3,681.0,France,Male,28.0,6.0,0.00,2.0,1.0,0.0,14081.64,0.0
4,587.0,France,Female,27.0,5.0,0.00,2.0,1.0,0.0,158958.90,0.0
...,...,...,...,...,...,...,...,...,...,...,...
14995,581.0,France,Male,35.0,1.0,0.00,2.0,1.0,0.0,161653.50,0.0
14996,806.0,Germany,Male,42.0,1.0,129120.64,2.0,1.0,1.0,161642.08,0.0
14997,620.0,France,Female,29.0,7.0,0.00,2.0,1.0,1.0,161579.85,0.0


In [ ]:
one_hot = OneHotEncoder(dtype=np.int8, sparse_output=False ) #drop = 'first' ma'lumot kamligi uchun shart emas
stan_scal = StandardScaler()
col_trans = ColumnTransformer([
    ('one_hot', one_hot, ['Geography', 'Gender'])
], remainder='passthrough').set_output(transform='pandas')

In [ ]:
col_trans

ColumnTransformer(remainder='passthrough',
                  transformers=[('one_hot',
                                 OneHotEncoder(dtype=<class 'numpy.int8'>,
                                               sparse_output=False),
                                 ['Geography', 'Gender'])])

In [ ]:
trans_df = col_trans.fit_transform(train)
test_df = col_trans.fit_transform(test)
trans_df

,one_hot__Geography_France,one_hot__Geography_Germany,one_hot__Geography_Spain,one_hot__Gender_Female,one_hot__Gender_Male,remainder__CreditScore,remainder__Age,remainder__Tenure,remainder__Balance,remainder__NumOfProducts,remainder__HasCrCard,remainder__IsActiveMember,remainder__EstimatedSalary,remainder__Exited
id,,,,,,,,,,,,,,
0,1,0,0,0,1,567.0,33.0,9.0,0.00,2.0,1.0,0.0,156792.89,0.0
1,1,0,0,1,0,628.0,38.0,3.0,0.00,1.0,1.0,1.0,51987.99,1.0
2,1,0,0,1,0,635.0,29.0,3.0,0.00,2.0,1.0,1.0,113079.19,0.0
3,1,0,0,0,1,681.0,28.0,6.0,0.00,2.0,1.0,0.0,14081.64,0.0
4,1,0,0,1,0,587.0,27.0,5.0,0.00,2.0,1.0,0.0,158958.90,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,1,0,0,0,1,581.0,35.0,1.0,0.00,2.0,1.0,0.0,161653.50,0.0
14996,0,1,0,0,1,806.0,42.0,1.0,129120.64,2.0,1.0,1.0,161642.08,0.0
14997,1,0,0,1,0,620.0,29.0,7.0,0.00,2.0,1.0,1.0,161579.85,0.0


In [ ]:
trans_df.describe()


,one_hot__Geography_France,one_hot__Geography_Germany,one_hot__Geography_Spain,one_hot__Gender_Female,one_hot__Gender_Male,remainder__CreditScore,remainder__Age,remainder__Tenure,remainder__Balance,remainder__NumOfProducts,remainder__HasCrCard,remainder__IsActiveMember,remainder__EstimatedSalary,remainder__Exited
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,0.602667,0.178667,0.218667,0.434800,0.565200,657.856800,37.710133,5.018667,42338.107539,1.590533,0.779133,0.496000,116944.059867,0.198467
std,0.489362,0.383086,0.413356,0.495747,0.495747,72.678739,8.144880,2.787407,59703.047751,0.525822,0.414845,0.500001,46047.485455,0.398859
min,0.000000,0.000000,0.000000,0.000000,0.000000,431.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,602.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,82644.332500,0.000000
50%,1.000000,0.000000,0.000000,0.000000,1.000000,661.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,122449.420000,0.000000
75%,1.000000,0.000000,0.000000,1.000000,1.000000,707.000000,42.000000,7.000000,109636.342500,2.000000,1.000000,1.000000,155703.022500,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,850.000000,72.000000,10.000000,187911.550000,5.000000,1.000000,1.000000,885120.790000,1.000000


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

In [ ]:
model_RF = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
model_GB = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model_KNN = KNeighborsClassifier(n_neighbors=5)
model_SVC = SVC(probability=True, kernel='rbf', C=1.0)
model_LR = LogisticRegression()

models = [model_LR, model_SVC, model_KNN, model_GB, model_RF]

In [ ]:
X_train = trans_df.drop(['remainder__Exited'], axis=1).copy()
y_train = trans_df['remainder__Exited'].copy()
X_test = test_df

In [ ]:
for model in models:
  model.fit(X_train, y_train)
  y_pred = model.predict(X_train)

  print(f"Model: {model} \n")
  print(f"Accuracy: {accuracy_score(y_train, y_pred)} \n")
  print(f"Classification Report: {classification_report(y_train, y_pred)}")
  print("  "*25)
  print("*"*25)
  print("  "*25)

Model: LogisticRegression() 

Accuracy: 0.8268 

Classification Report:               precision    recall  f1-score   support

         0.0       0.85      0.96      0.90     12023
         1.0       0.64      0.29      0.40      2977

    accuracy                           0.83     15000
   macro avg       0.74      0.62      0.65     15000
weighted avg       0.80      0.83      0.80     15000

                                                  
*************************
                                                  
Model: SVC(probability=True) 

Accuracy: 0.8015333333333333 

Classification Report:               precision    recall  f1-score   support

         0.0       0.80      1.00      0.89     12023
         1.0       0.00      0.00      0.00      2977

    accuracy                           0.80     15000
   macro avg       0.40      0.50      0.44     15000
weighted avg       0.64      0.80      0.71     15000

                                                  
**********

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model: KNeighborsClassifier() 

Accuracy: 0.8246666666666667 

Classification Report:               precision    recall  f1-score   support

         0.0       0.84      0.97      0.90     12023
         1.0       0.66      0.24      0.36      2977

    accuracy                           0.82     15000
   macro avg       0.75      0.61      0.63     15000
weighted avg       0.80      0.82      0.79     15000

                                                  
*************************
                                                  
Model: GradientBoostingClassifier(random_state=42) 

Accuracy: 0.9024666666666666 

Classification Report:               precision    recall  f1-score   support

         0.0       0.92      0.96      0.94     12023
         1.0       0.81      0.66      0.73      2977

    accuracy                           0.90     15000
   macro avg       0.87      0.81      0.83     15000
weighted avg       0.90      0.90      0.90     15000

                         

In [ ]:
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)),  # Cheklangan chuqurlik
    ('gb', GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)),  # Katta emas
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('svc', SVC(probability=True, kernel='rbf', C=1.0))
]

meta_model = LogisticRegression()

stacking_model = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=5)

In [ ]:
stacking_model.fit(X_train, y_train)

#train data uchun test
y_pred = stacking_model.predict(X_train)
accuracy = accuracy_score(y_train, y_pred)
print(f"Stacking model accuracy: {accuracy:.4f}")

Stacking model accuracy: 0.9022


In [ ]:
#Train uchun ROC AUC Score
y_proba_train = stacking_model.predict_proba(X_train)[:, 1]
roc_auc = roc_auc_score(y_train, y_proba_train)
print(f"ROC AUC Score: {roc_auc:.4f}")

ROC AUC Score: 0.9443


In [ ]:
y_stack_pred = stacking_model.predict_proba(X_test)[:, 1]
y_stack_pred

array([0.07344306, 0.12138651, 0.04162164, ..., 0.96657375, 0.03067923,
       0.0954939 ])

In [ ]:
#kaggle uchun csv ga aylantirish
ans = pd.DataFrame(y_stack_pred)

ans.columns = ['Exited']
ans['id'] = np.arange(15000, 25000 )
ans = ans[['id', 'Exited']]

ans.to_csv('predict_4.csv', index=False)
ans

,id,Exited
0,15000,0.073443
1,15001,0.121387
2,15002,0.041622
3,15003,0.087502
4,15004,0.030224
...,...,...
9995,24995,0.073750
9996,24996,0.055912
9997,24997,0.966574
9998,24998,0.030679
